# Change classification and Urban Change categories calculation
This notebook processes building count and height statistics to classify 8 types of urban change for the interval 2016-2023.

**Author:** Sai Ganesh Veeravalli  
**Dataset Used:** Google 2.5D Open Buildings (Temporal)  
**Analysis Year:** 2023  

# 📦 Section 1: Import Required Libraries

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os

## Section 1.1 Define input directory and ouput path

In [ ]:
input_dir = r"F:\DEPRIMAP\EARSEL2025\March2025\Data\csv_experiment"
output_path = os.path.join(input_dir, "urban_change_updated_23_16.gpkg")

# 🧭 Section 2: Building count and height change classification

## Section 2.1 Use 2016 and 2023 data

In [ ]:
# Only use 2016 and 2023 files
years = [16, 23]
gdfs = []

for year in years:
    path = os.path.join(input_dir, f"output_{year}.gpkg")
    gdf = gpd.read_file(path)
    gdf = gdf[["Org_FID", "geometry", f"count_{year}", f"mean_ht_{year}", f"median_ht_{year}"]]
    gdfs.append(gdf)

# Merge based on Org_FID
merged_gdf = gdfs[0]
merged_gdf = merged_gdf.merge(gdfs[1].drop(columns=["geometry"]), on="Org_FID", how="outer")
merged_gdf.set_geometry(gdfs[0].geometry, inplace=True)

## Sectio 2.2 Preprocessing and Difference calculation

In [ ]:
# 👉 If total building count is < 1 in any year, set count and heights to 0
# This ensures we don’t accidentally infer false differences from areas with no buildings
for year in years:
    count_col = f"count_{year}"
    mean_ht_col = f"mean_ht_{year}"
    median_ht_col = f"median_ht_{year}"
    merged_gdf.loc[merged_gdf[count_col] < 1, [count_col, mean_ht_col, median_ht_col]] = 0

# 👉 Calculate building count and height differences (2023 - 2016)
merged_gdf["ct_diff_23_16"] = merged_gdf["count_23"] - merged_gdf["count_16"]
merged_gdf["avg_ht_diff_23_16"] = merged_gdf["mean_ht_23"] - merged_gdf["mean_ht_16"]

## Section 2.3 Threshold definition

👉 Thresholds are based on literature and empirical distribution checks
- Count thresholds: Decrease < -1.0, Increase > 6.34 (IQR rule)
- Height thresholds: Decrease < -1.8, Increase > 1.8 (one floor ≈ 1.8m)

In [ ]:
def classify_count(x):
    if x < -1.0:
        return "Decrease"
    elif -1.0 <= x <= 6.34:
        return "Stable"
    else:
        return "Increase"

def classify_height(x):
    if x < -1.8:
        return "Decrease"
    elif -1.8 <= x <= 1.8:
        return "Stable"
    else:
        return "Increase"

# Apply classification to each grid
merged_gdf["class_count"] = merged_gdf["ct_diff_23_16"].apply(classify_count)
merged_gdf["class_avg_ht"] = merged_gdf["avg_ht_diff_23_16"].apply(classify_height)

# ⚙️ Section 3: Urban change categories calculation

## Section 3.1  Define urban change categories (typology) definitions

In [ ]:
# 👉 Combine count and height classes into 8 urban change categories

def classify_urban_change(row):
    c = row["class_count"]
    h = row["class_avg_ht"]
    if c == "Stable" and h == "Stable":
        return "Stable"
    elif c == "Stable" and h == "Increase":
        return "Vertical Densification"
    elif c == "Increase" and h == "Stable":
        return "Horizontal Densification"
    elif c == "Increase" and h == "Increase":
        return "Combined Densification"
    elif c == "Decrease" and h == "Decrease":
        return "Decline"
    elif c == "Increase" and h == "Decrease":
        return "Mixed Trend 1"
    elif c == "Decrease" and h == "Increase":
        return "Mixed Trend 2"
    elif (c == "Stable" and h == "Decrease") or (c == "Decrease" and h == "Stable"):
        return "Partial Decline"
    else:
        return "Undefined"

merged_gdf["urban_change"] = merged_gdf.apply(classify_urban_change, axis=1)

## Section 3.2 Export results

In [ ]:
# Save as GeoPackage for downstream use
merged_gdf.to_file(output_path, driver="GPKG")
print(f"🚀 Urban change classification saved to {output_path}")